In [ ]:
%%HTML
<style>
    div#notebook-container    { width: 100%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

In [ ]:
import sys

# データセットを配置したディレクトリのパスを設定
if 'google.colab' in sys.modules:
    dataset_dir = f"{mount_dir}/MyDrive/JPX_competition/data_dir_comp2"
else:
    dataset_dir = "/notebook/data_dir_comp2"

In [ ]:
from ast import literal_eval
from collections import Counter
from itertools import chain

import pandas as pd
import japanize_matplotlib

In [ ]:
%matplotlib inline

## データ読込

In [ ]:
# 入力パラメーターを設定します。ランタイム環境での実行時と同一フォーマットにします
inputs = {
    "stock_list": f"{dataset_dir}/stock_list.csv.gz",
    "stock_price": f"{dataset_dir}/stock_price.csv.gz",
    "stock_fin": f"{dataset_dir}/stock_fin.csv.gz",
    "stock_fin_price": f"{dataset_dir}/stock_fin_price.csv.gz",
    # ニュースデータ
    "tdnet": f"{dataset_dir}/tdnet.csv.gz",
    "disclosureItems": f"{dataset_dir}/disclosureItems.csv.gz",
    "nikkei_article": f"{dataset_dir}/nikkei_article.csv.gz",
    "article": f"{dataset_dir}/article.csv.gz",
    "industry": f"{dataset_dir}/industry.csv.gz",
    "industry2": f"{dataset_dir}/industry2.csv.gz",
    "region": f"{dataset_dir}/region.csv.gz",
    "theme": f"{dataset_dir}/theme.csv.gz",
    # 目的変数データ
    "stock_labels": f"{dataset_dir}/stock_labels.csv.gz",
}

In [ ]:
# 銘柄リストを取得
df_stock_list = pd.read_csv(inputs["stock_list"])
# 投資対象銘柄を取得
stock_codes = df_stock_list.loc[df_stock_list.loc[:, "universe_comp2"] == True, "Local Code"]

In [ ]:
# 日経電子版見出し・メタデータ読み込み
df_nikkei_article = pd.read_csv(inputs["nikkei_article"])
# インデックスを記事の掲載日に設定
df_nikkei_article.set_index("publish_datetime", inplace=True)
# インデックスを日付型に変換
df_nikkei_article.index = pd.to_datetime(df_nikkei_article.index)
# インデックスで安定ソート
df_nikkei_article.sort_index(kind="mergesort", inplace=True)

In [ ]:
# 適時開示資料のメタデータ読み込み
df_tdnet = pd.read_csv(inputs["tdnet"])
# インデックスを開示日時に設定
df_tdnet.index = pd.to_datetime(df_tdnet["disclosedDate"].astype(str) + " " + df_tdnet["disclosedTime"])
# インデックスで安定ソート
df_tdnet.sort_index(kind="mergesort", inplace=True)

In [ ]:
# 公開項目コード一覧を読み込み
df_disclosureitems = pd.read_csv(inputs["disclosureItems"])

## 日経電子版見出し・メタデータ

In [ ]:
# 件数を確認
df_nikkei_article.info()

In [ ]:
# データの開始日と内容を確認
df_nikkei_article.head(1).T

In [ ]:
# データの最終日を確認
df_nikkei_article.tail(1).T

In [ ]:
# 見出しの文字数の分布をプロット
df_nikkei_article.headline.str.len().hist(figsize=(10,10), bins=50)

In [ ]:
# 銘柄コードが含まれているデータを確認
df_nikkei_article.loc[~df_nikkei_article.loc[:, "company_g.stock_code"].isnull()].head().T

In [ ]:
# 月曜日を開始日として、週次の記事件数と銘柄コードを含む記事件数を取得
df_weekly_count = df_nikkei_article.resample("W-MON", label="left", closed="left")[["article_id", "company_g.stock_code"]].count()

In [ ]:
# 集計内容を確認
df_weekly_count.head(3)

In [ ]:
# 週次件数の統計量を確認
df_weekly_count.describe()

In [ ]:
#  週次件数の分布を確認
df_weekly_count.hist(figsize=(20, 10), alpha=0.5, bins=25)

In [ ]:
# プロット
ax = df_weekly_count.plot(figsize=(20, 8))
# グリッド設定
ax.grid(True)

In [ ]:
# 記事に記載されている銘柄コードを取得
s_stocks = df_nikkei_article.loc[~df_nikkei_article.loc[:, "company_g.stock_code"].isnull(), "company_g.stock_code"].str.split("\n")

In [ ]:
# 銘柄コード別の件数を取得
s_stock_counts = pd.Series(Counter(chain.from_iterable(s_stocks))).sort_values(ascending=False)

In [ ]:
# 投資対象銘柄に絞り込み
s_stock_counts = s_stock_counts.loc[s_stock_counts.index.astype(int).isin(stock_codes)]

In [ ]:
# データフレームに変換して、インデックスをリセット
df_stock_counts = s_stock_counts.to_frame().reset_index(drop=True)
# カラム名を設定
df_stock_counts.rename(columns={0: "article_count"}, inplace=True)
# 投資対象銘柄全体におけるdisclosureItems別の件数の割合を算出
df_stock_counts.loc[:, "percentage"] = (df_stock_counts["article_count"] / df_stock_counts["article_count"].sum()) * 100
# 件数割合の累積を計算
df_stock_counts.loc[:, "cumulative_percentage"] = df_stock_counts["percentage"].cumsum()
# 投資対象銘柄コード別の件数をプロット (全銘柄)
ax = df_stock_counts[["article_count"]].plot(figsize=(20, 8))
#  グリッド設定
ax.grid(True)
# 凡例を左上に表示
ax.legend(loc="upper left")
# 2つ目のy軸を作成
ax2 = ax.twinx()
# 開示件数割合の累積をプロット
df_stock_counts[["cumulative_percentage"]].reset_index(drop=True).plot(ax=ax2, color="orange")
# 凡例を右上に表示
ax2.legend(loc="upper right")

In [ ]:
# 投資対象銘柄コード別の記事件数 (件数上位50銘柄)
ax = s_stock_counts.head(50).plot.bar(figsize=(20, 8))
#  グリッド設定
ax.grid(True)

## 適時開示資料のメタデータを確認

In [ ]:
# 件数を確認
df_tdnet.info()

In [ ]:
# データの開始日と内容を確認
df_tdnet.head(1).T

In [ ]:
# データの最終日を確認
df_tdnet.tail(1).T

In [ ]:
# 銘柄コードを4桁に変更
df_tdnet.loc[:, "stock_code"] = df_tdnet.loc[:, "code"].astype(str).str[:4].astype(int)

In [ ]:
# 投資対象銘柄に絞り込み
filter_universe = df_tdnet.loc[:, "stock_code"].isin(stock_codes)

In [ ]:
# 月曜日を開始日として、投資対象銘柄の週次の件数を取得
df_tdnet_weekly_count = df_tdnet.loc[filter_universe].resample("W-MON", label="left", closed="left")[["disclosureNumber"]].count()

In [ ]:
# 集計内容を確認
df_tdnet_weekly_count.head(3)

In [ ]:
# プロット
ax = df_tdnet_weekly_count.plot(figsize=(20, 8))
# グリッド設定
ax.grid(True)

In [ ]:
#  資対象銘柄の週次件数の分布を確認
df_tdnet_weekly_count.hist(figsize=(10, 10))

In [ ]:
# 投資対象銘柄のdisclosureItemsをリスト形式のシリーズとして取得
s_disclosureItems = df_tdnet.loc[filter_universe, "disclosureItems"].apply(literal_eval)

In [ ]:
s_disclosureItems[:10]

In [ ]:
# 投資対象銘柄のdisclosureItems別の件数を取得
s_disclosureitems_count = pd.Series(Counter(chain.from_iterable(s_disclosureItems)))
# カラム名を設定
s_disclosureitems_count.name = "disclosureItems_count"
# 公開項目コードをint型に変更
s_disclosureitems_count.index = s_disclosureitems_count.index.astype(int)

In [ ]:
# 項目の日本語名を表示するために適時開示資料の公開項目コードと結合
df_disclosureitems_with_label = pd.merge(s_disclosureitems_count, df_disclosureitems, left_index=True, right_on=["公開項目コード"])
# indexを設定
df_disclosureitems_with_label.set_index("コード値定義", inplace=True)

In [ ]:
# 投資対象銘柄のdisclosureItems別の件数を多い順に並び替え
df_count_by_disclosureitems_with_label = df_disclosureitems_with_label.loc[:, ["disclosureItems_count"]].sort_values("disclosureItems_count", ascending=False).reset_index(drop=True)
# 投資対象銘柄全体におけるdisclosureItems別の件数の割合を算出
df_count_by_disclosureitems_with_label.loc[:, "percentage"] = (df_count_by_disclosureitems_with_label["disclosureItems_count"] / df_count_by_disclosureitems_with_label["disclosureItems_count"].sum()) * 100
# 件数割合の累積を計算
df_count_by_disclosureitems_with_label.loc[:, "cumulative_percentage"] = df_count_by_disclosureitems_with_label["percentage"].cumsum()
# プロット (全て)
ax = df_count_by_disclosureitems_with_label[["disclosureItems_count"]].plot(figsize=(20, 8))
#  グリッド設定
ax.grid(True)
# 凡例を左上に表示
ax.legend(loc="upper left")
# 2つ目のy軸を作成
ax2 = ax.twinx()
# 開示件数割合の累積をプロット
df_count_by_disclosureitems_with_label[["cumulative_percentage"]].reset_index(drop=True).plot(ax=ax2, color="orange")
# 凡例を右上に表示
ax2.legend(loc="upper right")

In [ ]:
# 投資対象銘柄のdisclosureItems別に件数を多い順にプロット (上位50項目)
ax = df_disclosureitems_with_label.loc[:, ["disclosureItems_count"]].sort_values("disclosureItems_count", ascending=False).head(50).plot(kind="bar", figsize=(20, 8))
#  グリッド設定
ax.grid(True)

In [ ]:
# 投資対象銘柄別に開示件数を集計して、開示件数の多い順に並び替え
df_tdnet_count_by_stock_code = df_tdnet.loc[filter_universe].groupby("stock_code")[["disclosureNumber"]].count().sort_values("disclosureNumber", ascending=False)
# 投資対象銘柄全体における開示件数の割合を集計
df_tdnet_count_by_stock_code.loc[:, "percentage"] = (df_tdnet_count_by_stock_code["disclosureNumber"] / df_tdnet_count_by_stock_code["disclosureNumber"].sum()) * 100
# 開示件数割合の累積を計算
df_tdnet_count_by_stock_code.loc[:, "cumulative_percentage"] = df_tdnet_count_by_stock_code["percentage"].cumsum()
# 投資対象銘柄別に開示件数を多い順にプロット
ax = df_tdnet_count_by_stock_code[["disclosureNumber"]].reset_index(drop=True).plot(figsize=(20, 8))
# グリッド設定
ax.grid(True)
# 凡例を左上に表示
ax.legend(loc="upper left")
# 2つ目のy軸を作成
ax2 = ax.twinx()
# 開示件数割合の累積をプロット
df_tdnet_count_by_stock_code[["cumulative_percentage"]].reset_index(drop=True).plot(ax=ax2, color="orange")
# 凡例を右上に表示
ax2.legend(loc="upper right")

In [ ]:
# 投資対象銘柄別に開示件数を多い順にプロット (上位50銘柄)
ax = df_tdnet.loc[filter_universe].groupby("stock_code")[["disclosureNumber"]].count().sort_values("disclosureNumber", ascending=False).head(50).plot(kind="bar", figsize=(20, 8))
# 　グリッド設定
ax.grid(True)